# Make a table of Absorption lines

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"
import pandas as pd
import numpy as np
import astropy.table
from astropy.table import Table, vstack
from matplotlib import pyplot as plt

from pyigm.igm import igmsightline
from cgmsquared import io

import yaml

from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord

In [2]:
data_path = "/Users/mwilde/Dropbox/IGM_spectra/Completed_QSOs/"
abs_file = 'J1555+3628_werksquad.json'

In [3]:
data_path+abs_file

'/Users/mwilde/Dropbox/IGM_spectra/Completed_QSOs/J1555+3628_werksquad.json'

### Read in the json file

this instantiates an IGMSightline

In [4]:
field = igmsightline.IGMSightline.from_json(data_path+abs_file)

read_sets: Using set file -- 
  /Users/mwilde/linetools/linetools/lists/sets/llist_v1.2.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them


### Make the igmsystems

Use the components from the json to generate a list of IGMSystems (AbsSystems from AbsComponent objects)

"Build a list of AbsSystems from a list of AbsComponents"

igmsys is actually an IGMsys (subclass of AbsSystems)
each AbsSystem has components which each have individual abslines

In [5]:
igmsys = field.make_igmsystems()

### Accessing the components and elements is not super straight forward

In [6]:
igmsys[0]._components[0]._abslines[0].wrest

<Quantity 1548.204 Angstrom>

In [51]:
line = igmsys[0]._components[0]._abslines[0]
line.name

'CIV 1548'

In [38]:
comp = igmsys[0]._components[0]

### AbsComponents are made up of AbsLines, AbsSystems have the same wavelength (?)

In [10]:
absline = igmsys[0]._components[0]._abslines[0]
igmsys[0].zabs, absline.z, comp.zcomp

(-0.0005575197123075295, -0.00042, -0.0005575197123075295)

## Need to output a table specifically for Jess' IDL routine

In [11]:
wrest_list = []
name_list = []
z_abs_list = []
z_line_list = []
reliability_list = []
print("number of AbsSystems:", len(igmsys))
for abssys in igmsys:
    # print("\t num comps", len(components_list))
    components_list = abssys._components
    z_abs = abssys.zabs
    for comp in components_list:
        # print("\t \t num abslines",len(absline_list))
        reliability = comp.reliability
        absline_list = comp._abslines
        for absline in absline_list:
            # print(absline.wrest.value, absline.name, absline.z, z_abs)
            wrest_list.append(absline.wrest.value)
            name_list.append(absline.name)
            z_abs_list.append(z_abs)
            z_line_list.append(absline.z)
            reliability_list.append(reliability)


number of AbsSystems: 91


In [13]:
# Make a table
absline_table = Table(data=[wrest_list, name_list, z_abs_list, z_line_list, reliability_list], 
      names=["wrest", "name", "z_abssys", "z_absline", "reliability_list"])

In [49]:
absline_table[:5]

wrest,name,z_abssys,z_absline,reliability_list
float64,str10,float64,float64,str1
1548.204,CIV 1548,-0.0005575197123075295,-0.00042,a
1550.781,CIV 1550,-0.0005575197123075295,-0.00042,a
1393.7602,SiIV 1393,-0.0005575197123075295,-0.00043486369472197473,a
1402.7729,SiIV 1402,-0.0005575197123075295,-0.00043486369472197473,a
1548.204,CIV 1548,-0.0005575197123075295,-0.00024,a


In [44]:
absline_table.write('absorber_info_for_jess.lst', format='ascii')

# The Following is to get a rough estimate of the Column density to make a quick plot. 

Since these are raw werksquad files, we need to artificially set the error and the flag to get a column density

This part is from a suggestion from X.

In [56]:
ion_species = []
for a in igmsys:
    for c in a._components:
        for absline in c._abslines:
            absline.attrib['flag_N'] = 1
            absline.attrib['sig_N'] = np.array([1e13,1e13])/u.cm**2
        c.synthesize_colm(overwrite=True)
    a.update_component_colm(overwrite=True)
    
    # Fills the ionN Table from the list of components
    a.fill_ionN()
    ion_species.append(a._ionN)
    

### Make a table of all the absorption systems

In [59]:
abssysTable = vstack(ion_species)
abssysTable

RA,DEC,comp_name,Z,ion,Ej,vmin,vmax,ion_name,flag_N,logN,sig_logN [2],b,sig_b,vel,sig_vel,comment,reliability,z_sys
deg,deg,,,,1 / cm,km / s,km / s,,,,,km / s,km / s,km / s,km / s,,,
float64,float64,str15,int64,int64,float64,float64,float64,str5,int64,float64,float64,float64,float64,float64,float64,str10,str1,float64
238.76833,36.48001,z-0.00056_CIV,6,4,0.0,-73.97247607447584,73.97247607455371,CIV,1,13.9,0.0386606644156722 .. 0.0386606644156722,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00053_SiIV,14,4,0.0,-60.82181366133387,60.82181366130564,SiIV,1,13.12684879599214,0.22930869221109298 .. 0.22930869221109298,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00029_CIV,6,4,0.0,-136.4381225374798,136.4381225375255,CIV,1,14.5,0.00971111983788723 .. 0.00971111983788723,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00029_AlII,13,2,0.0,-88.76697128943584,88.76697128885249,AlII,1,13.915186712087836,0.052795584122677534 .. 0.052795584122677534,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00023_CII,6,2,0.0,-121.64362732256411,121.64362732251409,CII,1,15.062052035971078,0.0037647164816175535 .. 0.0037647164816175535,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00023_SiII,14,2,0.0,-167.67094576892848,167.67094576895997,SiII,1,14.95,0.002179211134009477 .. 0.002179211134009477,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00022_FeII,26,2,0.0,-141.3696209424226,141.36962094243484,FeII,1,15.0,0.0017729997979489165 .. 0.0017729997979489165,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00022_SiIII,14,3,0.0,-162.739447364004,162.73944736394836,SiIII,1,14.00309128783129,0.04312141698360396 .. 0.04312141698360396,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295


In [61]:
group_by_z = abssysTable.group_by('z_sys')
group_by_z

RA,DEC,comp_name,Z,ion,Ej,vmin,vmax,ion_name,flag_N,logN,sig_logN [2],b,sig_b,vel,sig_vel,comment,reliability,z_sys
deg,deg,,,,1 / cm,km / s,km / s,,,,,km / s,km / s,km / s,km / s,,,
float64,float64,str15,int64,int64,float64,float64,float64,str5,int64,float64,float64,float64,float64,float64,float64,str10,str1,float64
238.76833,36.48001,z-0.00056_CIV,6,4,0.0,-73.97247607447584,73.97247607455371,CIV,1,13.9,0.0386606644156722 .. 0.0386606644156722,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00053_SiIV,14,4,0.0,-60.82181366133387,60.82181366130564,SiIV,1,13.12684879599214,0.22930869221109298 .. 0.22930869221109298,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00029_CIV,6,4,0.0,-136.4381225374798,136.4381225375255,CIV,1,14.5,0.00971111983788723 .. 0.00971111983788723,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00029_AlII,13,2,0.0,-88.76697128943584,88.76697128885249,AlII,1,13.915186712087836,0.052795584122677534 .. 0.052795584122677534,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00023_CII,6,2,0.0,-121.64362732256411,121.64362732251409,CII,1,15.062052035971078,0.0037647164816175535 .. 0.0037647164816175535,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00023_SiII,14,2,0.0,-167.67094576892848,167.67094576895997,SiII,1,14.95,0.002179211134009477 .. 0.002179211134009477,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00022_FeII,26,2,0.0,-141.3696209424226,141.36962094243484,FeII,1,15.0,0.0017729997979489165 .. 0.0017729997979489165,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00022_SiIII,14,3,0.0,-162.739447364004,162.73944736394836,SiIII,1,14.00309128783129,0.04312141698360396 .. 0.04312141698360396,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295


In [62]:
print(abssysTable['ion_name', 'logN', 'z_sys'])

ion_name        logN                z_sys         
-------- ------------------ ----------------------
     CIV               13.9 -0.0005575197123075295
    SiIV  13.12684879599214 -0.0005575197123075295
     CIV               14.5 -0.0005575197123075295
    AlII 13.915186712087836 -0.0005575197123075295
     CII 15.062052035971078 -0.0005575197123075295
    SiII              14.95 -0.0005575197123075295
    FeII               15.0 -0.0005575197123075295
   SiIII  14.00309128783129 -0.0005575197123075295
    NiII 13.830159412156343 -0.0005575197123075295
    SiIV 13.599368555373001 -0.0005575197123075295
     ...                ...                    ...
    CIII  13.64244658041559     0.6022781134663002
     ClV              12.83     0.6022781134663002
     NIV               13.9     0.6022781134663002
    NIII               13.8     0.6022781134663002
     OVI 14.444301451936564     0.6022781134663002
      NI               14.5     0.6022781134663002
      HI 15.130957792864205    

# or load it from cgmsquared

In [63]:
import cgmsquared

In [64]:
cgmsquared.utils.make_abssys_table(data_path+abs_file)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them
AbsComp coordinates do not match.  Best to set them


RA,DEC,comp_name,Z,ion,Ej,vmin,vmax,ion_name,flag_N,logN,sig_logN [2],b,sig_b,vel,sig_vel,comment,reliability,z_sys
deg,deg,,,,1 / cm,km / s,km / s,,,,,km / s,km / s,km / s,km / s,,,
float64,float64,str15,int64,int64,float64,float64,float64,str5,int64,float64,float64,float64,float64,float64,float64,str10,str1,float64
238.76833,36.48001,z-0.00056_CIV,6,4,0.0,-73.97247607447584,73.97247607455371,CIV,1,13.9,0.0386606644156722 .. 0.0386606644156722,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00053_SiIV,14,4,0.0,-60.82181366133387,60.82181366130564,SiIV,1,13.12684879599214,0.22930869221109298 .. 0.22930869221109298,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00029_CIV,6,4,0.0,-136.4381225374798,136.4381225375255,CIV,1,14.5,0.00971111983788723 .. 0.00971111983788723,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00029_AlII,13,2,0.0,-88.76697128943584,88.76697128885249,AlII,1,13.915186712087836,0.052795584122677534 .. 0.052795584122677534,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00023_CII,6,2,0.0,-121.64362732256411,121.64362732251409,CII,1,15.062052035971078,0.0037647164816175535 .. 0.0037647164816175535,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00023_SiII,14,2,0.0,-167.67094576892848,167.67094576895997,SiII,1,14.95,0.002179211134009477 .. 0.002179211134009477,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00022_FeII,26,2,0.0,-141.3696209424226,141.36962094243484,FeII,1,15.0,0.0017729997979489165 .. 0.0017729997979489165,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
238.76833,36.48001,z-0.00022_SiIII,14,3,0.0,-162.739447364004,162.73944736394836,SiIII,1,14.00309128783129,0.04312141698360396 .. 0.04312141698360396,0.0,0.0,0.0,0.0,,a,-0.0005575197123075295
